In [1]:
# Imports

import numpy as np
import pandas as pd

from math import ceil

import string
import re

import csv

import time

import valo_api as va

In [2]:
## For a given match_id, generate a match_dict for performances

def get_performance_data(match_response):
    
    ## Calculate location in the kill list
    kill_ctr = 0 
    
    j = 0
    
    match_dict = {}

    for player in match_response.players.all_players:

        try:
            headshot_percentage = 100 * player.stats.headshots / (player.stats.headshots + player.stats.bodyshots + player.stats.legshots)
        except ZeroDivisionError:
            headshot_percentage = 0
    
        
        player_dict = {"Name": player.name,
                       "Agent": player.character,
                       "ACS": player.stats.score / match_response.metadata.rounds_played,
                       "Kills": player.stats.kills,
                       "Deaths": player.stats.deaths,
                       "Assists": player.stats.assists,
                       "KAST": 0,
                       "ADR": player.damage_made / match_response.metadata.rounds_played,
                       "HS%": headshot_percentage,
                       "FK": 0,
                       "FD": 0,
                       "Team": player.team,
                       "Judge Kills": 0,
                       "OP Kills": 0,
                       "1v2s":0,
                       "1v2s won":0,
                       "1v3s":0,
                       "1v3s won":0,
                       "1v4s":0,
                       "1v4s won":0,
                       #"1v5s":0,
                       #"1v5s won":0,
                       "Match ID": match_response.metadata.matchid}

        match_dict[player.puuid] = player_dict

    ## Calculating first deaths / first kills / KAST

    for rd in match_response.rounds:
        
        earliest_kill_time = 10000000

        kill_events = []
        
        j += 1
        
        for player in rd.player_stats:

            for kill_event in player.kill_events: 

                kill_events.append(kill_event)

                ## Identify FK / FD

                if kill_event.kill_time_in_round < earliest_kill_time:

                    earliest_kill_time = kill_event.kill_time_in_round
                    first_killer = kill_event.killer_puuid
                    first_victim = kill_event.victim_puuid
                    
                ## Calculate Operator
                if kill_event.damage_weapon_name == 'Operator':
                    match_dict[kill_event.killer_puuid]['OP Kills'] += 1
                    
                 ## Calculate judge kills
                if kill_event.damage_weapon_name == 'Judge':
                    match_dict[kill_event.killer_puuid]['Judge Kills'] += 1
            
        clutch_player = None
        clutch_case = 'Undecided'
        
        ## here we are using match_response.kills because it is ordered correctly: 
        
        ## Red team alive
        red_team = match_response.players.red.copy()
        
        # Blue team alive
        blue_team = match_response.players.blue.copy()
        
        for i in range(len(kill_events)):
            
            kill_event = match_response.kills[kill_ctr]
            kill_ctr += 1
            
            if kill_event.victim_team == 'Blue':
                
                    ## Loop through current blue team members
                for player in blue_team:
                        
                    ## If member ID matches victim ID
                    if player.puuid == kill_event.victim_puuid:
                        ## Remove victim from living members
                        blue_team.remove(player)
            else:
                for player in red_team:
                        
                    ## If member ID matches victim ID
                    if player.puuid == kill_event.victim_puuid:
                        ## Remove victim from living members
                        red_team.remove(player)
                    
                ## Check if Blue team only has one survivor at this point
                
            if len(blue_team) == 1 and clutch_case == 'Undecided':
                clutch_case = f'1v{len(red_team)}'
                clutch_player = blue_team[0]
                match_dict[clutch_player.puuid]


            if len(red_team) == 1 and clutch_case == 'Undecided':
                clutch_case = f'1v{len(blue_team)}'
                clutch_player = red_team[0]
        
        ## Check if the team clutched the round
        if clutch_player != None:
             
            if clutch_case == '1v2':
                match_dict[clutch_player.puuid]['1v2s'] += 1
                match_dict[clutch_player.puuid]['1v2s won'] += (clutch_player.team == rd.winning_team)
                    
            if clutch_case == '1v3':
                match_dict[clutch_player.puuid]['1v3s'] += 1
                match_dict[clutch_player.puuid]['1v3s won'] += (clutch_player.team == rd.winning_team)
                    
            if clutch_case == '1v4':
                match_dict[clutch_player.puuid]['1v4s'] += 1
                match_dict[clutch_player.puuid]['1v4s won'] += (clutch_player.team == rd.winning_team)
                        

        for player in rd.player_stats: ## Checking for KAST

            KAST = False
            survived = True

            if player.kills > 0: ## If player has kills:
                KAST = True

            for kill_event in kill_events: ## Check if player has assists

                for assistant in kill_event.assistants:
                    if player.player_puuid == assistant.assistant_puuid:
                        KAST = True
                        break

            for kill_event in kill_events: ## Identify the player's killer

                if kill_event.victim_puuid == player.player_puuid:

                    survived = False

                    killer = kill_event.killer_puuid
                    killtime = kill_event.kill_time_in_round

                    for kill_event in kill_events: ## Check if that killer got killed 5 seconds after the time
                        if (kill_event.victim_puuid == killer and kill_event.kill_time_in_round < killtime + 5000):
                            KAST = True

            if survived:
                KAST = True

            if KAST:
                match_dict[player.player_puuid]['KAST'] += 1 / match_response.metadata.rounds_played



        ## Update player dictionary with FK / FD information

        match_dict[first_killer]['FK'] += 1
        match_dict[first_victim]['FD'] += 1
        
        ## Check if the team won the round with man advantage
   
    return match_dict

In [3]:
match_response = va.get_match_details_v2('da1162ae-5d65-42fd-8c48-33f075c957b1')
get_performance_data(match_response)

{'dc6aa7e2-491c-5ca7-ad34-5c8f655f8e6c': {'Name': 'Kyrigan',
  'Agent': 'Brimstone',
  'ACS': 291.2608695652174,
  'Kills': 22,
  'Deaths': 13,
  'Assists': 13,
  'KAST': 0.869565217391304,
  'ADR': 191.7826086956522,
  'HS%': 40.0,
  'FK': 3,
  'FD': 2,
  'Team': 'Red',
  'Judge Kills': 0,
  'OP Kills': 0,
  '1v2s': 1,
  '1v2s won': 1,
  '1v3s': 0,
  '1v3s won': 0,
  '1v4s': 0,
  '1v4s won': 0,
  'Match ID': 'da1162ae-5d65-42fd-8c48-33f075c957b1'},
 '05ff3ccc-227f-56f0-ab05-4c36536ccb90': {'Name': 'Prophesie',
  'Agent': 'Neon',
  'ACS': 209.1304347826087,
  'Kills': 17,
  'Deaths': 17,
  'Assists': 3,
  'KAST': 0.652173913043478,
  'ADR': 131.04347826086956,
  'HS%': 11.428571428571429,
  'FK': 4,
  'FD': 3,
  'Team': 'Blue',
  'Judge Kills': 0,
  'OP Kills': 0,
  '1v2s': 1,
  '1v2s won': 0,
  '1v3s': 1,
  '1v3s won': 0,
  '1v4s': 0,
  '1v4s won': 0,
  'Match ID': 'da1162ae-5d65-42fd-8c48-33f075c957b1'},
 '49a7d364-562c-5fce-86e9-9969a0ac8730': {'Name': 'aT Kyshin',
  'Agent': 'Skye'

In [4]:
def get_match_data(match_response):

    match_dict = {
        "Match ID": match_response.metadata.matchid,
        "Map": match_response.metadata.map,
        "Total Rounds": match_response.metadata.rounds_played,
        "Team Blue Rounds": match_response.teams.blue.rounds_won,
        "Team Red Rounds": match_response.teams.red.rounds_won,
        "Team Blue Win?": match_response.teams.blue.has_won,
        "Team Blue Attack Rounds":0,
        "Team Blue Defense Rounds":0,
        "Team Red Attack Rounds":0,
        "Team Red Defense Rounds":0,
        "Team Blue 5v4s":0,
        "Team Blue 4v5s":0,
        "Team Blue 5v4s won":0,
        "Team Blue 4v5s won":0,
        "Team Red 5v4s":0,
        "Team Red 4v5s":0,
        "Team Red 5v4s won":0,
        "Team Red 4v5s won":0
    }
    
    ## From the API, red team always starts on attack
    
    for count, rnd in enumerate(match_response.rounds):
        
        if count < 12: ## For the first half, red team is attackers
            if rnd.winning_team == 'Red':
                match_dict['Team Red Attack Rounds'] += 1
            else:
                match_dict['Team Blue Defense Rounds'] += 1
        else:
            if rnd.winning_team == 'Red':
                match_dict['Team Red Defense Rounds'] += 1
            else:
                match_dict['Team Blue Attack Rounds'] += 1
                
                
        ## Identify which team secured first kill - and see if they convered it
        
        earliest_kill_time = 10000000
        kill_events = []

        for player in rnd.player_stats:

            for kill_event in player.kill_events: 
                
                kill_events.append(kill_event)
                
                if kill_event.kill_time_in_round < earliest_kill_time:

                    earliest_kill_time = kill_event.kill_time_in_round
                    fk_team = kill_event.killer_team
                
        if fk_team == 'Blue':    
            match_dict['Team Blue 5v4s'] += 1
            match_dict['Team Red 4v5s'] += 1    
            
            if rnd.winning_team == 'Blue':
                match_dict['Team Blue 5v4s won'] += 1
            else:
                match_dict['Team Red 4v5s won'] += 1
        
        else:
            match_dict['Team Blue 4v5s'] += 1
            match_dict['Team Red 5v4s'] += 1    
            
            if rnd.winning_team == 'Blue':
                match_dict['Team Blue 4v5s won'] += 1
            else:
                match_dict['Team Red 5v4s won'] += 1
        
    return match_dict

In [5]:

def clean_performance_df(match_response, team_blue, team_red): 
    
    ## Extract performance data and place it into a dataframe
    df = pd.DataFrame.from_dict(get_performance_data(match_response), orient='index')
    df.index = df.index.set_names('player_id')
    
    ## Rename 'blue' and 'red' to proper team names
    
    df['Team'] = df['Team'].apply(lambda x: team_blue if x == 'Blue' else team_red)
    
    ## Conver KAST to percentage
    
    df['KAST'] = df['KAST'] * 100
    
    ## Round
    
    df = df.round(1)
    
    ## Write down performance data
    df.to_csv('data/performances.csv', mode='a', header=False)


def clean_match_df(match_response, team_blue, team_red):
    
    ## Get dict
    match_dict = get_match_data(match_response)
    
    ## Data cleaning
    match_dict['Team Red'] = team_red
    match_dict['Team Blue'] = team_blue
    match_dict['Winner'] = team_blue if match_dict['Team Blue Win?'] else team_red
    match_dict.pop('Team Blue Win?')
    
    ## Write general match data to CSV
    
    
    
    with open('data/maps.csv', 'a') as csv_file:  
        
        for key, value in match_dict.items():
        
            csv_file.write(f"{str(value)},")
        csv_file.write("\n")
    


In [6]:
maps_df = pd.read_csv('data/maps.csv',usecols=[0],names=['match_id'],header=None)


In [7]:

## Get match database


with open('data/all_matches.csv', 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')

    # Skip header row
    next(reader)
    
    
    
    for row in reader:
        
        ## Check if this row's match ID already exists in the maps.csv
        
        if maps_df['match_id'].str.contains(row[0]).any():
            continue
        
        ## Get match response
        try:
            match_response = va.get_match_details_v2(row[0])
        except: ## Case of error
            print(f"Match ID {row[0]} not found, skipping to next match:")
            continue
            
            
        ## Get general match data    
        clean_match_df(match_response, row[1], row[2])
            
        ## Get performances
        clean_performance_df(match_response, row[1], row[2])
        
        
        ## Give a second so not to overwork the API
        time.sleep(1)
        

Match ID bf87bcc5-bf2b-4f2b-ba67-663b50628193 not found, skipping to next match:
Match ID 1354f97a-5c69-4b97-92f9-480b2e9e00cf not found, skipping to next match:
Match ID c472e91d-93e8-4c94-bf1c-dc223c673616 not found, skipping to next match:


In [8]:
performance_header = ['player_id','name','agent','acs','kills','deaths','assists','kast','adr','hs','fk','fd','team','judge_kills','op_kills','1v2s','1v2s_won','1v3s','1v3s_won','1v4s','1v4s_won','match_id']

performances_df = pd.read_csv('data/performances.csv',names=performance_header, header=None)

performances_df.head()

,player_id,name,agent,acs,kills,deaths,assists,kast,adr,hs,...,team,judge_kills,op_kills,1v2s,1v2s_won,1v3s,1v3s_won,1v4s,1v4s_won,match_id
0,dc6aa7e2-491c-5ca7-ad34-5c8f655f8e6c,Kyrigan,Brimstone,291.3,22,13,13,87.0,191.8,40.0,...,Easy As Pie,0,0,1,1,0,0,0,0,da1162ae-5d65-42fd-8c48-33f075c957b1
1,05ff3ccc-227f-56f0-ab05-4c36536ccb90,Prophesie,Neon,209.1,17,17,3,65.2,131.0,11.4,...,Alpha Threat,0,0,1,0,1,0,0,0,da1162ae-5d65-42fd-8c48-33f075c957b1
2,49a7d364-562c-5fce-86e9-9969a0ac8730,aT Kyshin,Skye,138.6,11,17,6,52.2,88.3,15.0,...,Alpha Threat,0,0,0,0,1,0,3,0,da1162ae-5d65-42fd-8c48-33f075c957b1
3,22276eb0-e1ec-5702-80f3-9985577d4e82,AnimateDragon,Brimstone,291.9,24,17,6,69.6,176.5,22.8,...,Alpha Threat,0,6,0,0,1,0,0,0,da1162ae-5d65-42fd-8c48-33f075c957b1
4,dae24c00-bf37-576b-b518-3b2abc60bf76,f a b i o,Yoru,266.8,25,13,5,82.6,172.2,22.1,...,Easy As Pie,0,0,0,0,1,0,0,0,da1162ae-5d65-42fd-8c48-33f075c957b1


In [9]:
map_header = ['match_id','map','rounds_played','team_blue_rounds','team_red_rounds',
              'team_blue_attack','team_blue_defense','team_red_attack','team_red_defense',"blue_5v4s",
        "blue_4v5s",
        "blue_5v4s_won",
        "blue_4v5s_won",
        "red_5v4s",
        "red_4v5s",
        "red_5v4s_won",
        "red_4v5s_won",'team_blue','team_red','winner','dummy']



maps_df = pd.read_csv('data/maps.csv',names=map_header, header=None)

maps_df.drop('dummy', axis=1,inplace=True)


maps_df.shape

(38, 20)

In [10]:
total_rounds = maps_df[maps_df['team_blue'] == 'Easy As Pie']['rounds_played'].sum() + \
               maps_df[maps_df['team_red'] == 'Easy As Pie']['rounds_played'].sum()

## Calculate 4v5 winrate

total_disadvantange = maps_df[maps_df['team_blue'] == 'Easy As Pie']['blue_4v5s'].sum() + \
                   maps_df[maps_df['team_red'] == 'Easy As Pie']['red_4v5s'].sum()


disadvantage_won = maps_df[maps_df['team_blue'] == 'Easy As Pie']['blue_4v5s_won'].sum() + \
                   maps_df[maps_df['team_red'] == 'Easy As Pie']['red_4v5s_won'].sum()

print(f"Disadvantage rounds (4v5s): {total_disadvantange}")
print(f"Disadvantage rounds / Total rounds: {total_disadvantange/total_rounds}")
print(f"Disadvantage round success rate: {disadvantage_won/total_disadvantange * 100}%")

Disadvantage rounds (4v5s): 227
Disadvantage rounds / Total rounds: 0.5078299776286354
Disadvantage round success rate: 25.55066079295154%


In [11]:
## Calculate 5v4 winrate

total_advantange = maps_df[maps_df['team_blue'] == 'Easy As Pie']['blue_5v4s'].sum() + \
                   maps_df[maps_df['team_red'] == 'Easy As Pie']['red_5v4s'].sum()


advantage_won = maps_df[maps_df['team_blue'] == 'Easy As Pie']['blue_5v4s_won'].sum() + \
                   maps_df[maps_df['team_red'] == 'Easy As Pie']['red_5v4s_won'].sum()

print(f"Advantage rounds (5v4s): {total_advantange}")
print(f"Advantage rounds / Total rounds: {total_advantange/total_rounds}")
print(f"Advantage round success rate: {advantage_won/total_advantange * 100}%")

Advantage rounds (5v4s): 220
Advantage rounds / Total rounds: 0.49217002237136465
Advantage round success rate: 70.9090909090909%


In [12]:
## Identify players with most FK/FD

eap_df = performances_df[performances_df['team'] == 'Easy As Pie']

## Factor in number of rounds played

eap_df = eap_df.merge(maps_df)

## Replace 'AT Greensmurf' and 'I miss her' with 'Strawberry'

eap_df.replace('AT GreenSmurf32', 'StrawBerry', inplace=True)
eap_df.replace('i miss her', 'StrawBerry', inplace=True)

## Replace 'p o o j' with 'PAYGE'

eap_df.replace('p o o j', 'PAYGE', inplace=True)



def player_summary(df):
    ## Get player summary
    player_summary = df.groupby('name').agg(
                        hs = ('hs', np.mean),
                        op_kills = ('op_kills', np.sum),
                        judge_kills = ('judge_kills', np.sum),
                        acs = ('acs', np.mean),
                        kast = ('kast',np.mean),
                        total_fk=('fk', np.sum),
                        total_fd=('fd', np.sum),
                        total_1v2s=('1v2s',np.sum),
                        total_1v2s_won=('1v2s_won',np.sum),
                        total_1v3s=('1v3s',np.sum),
                        total_1v3s_won=('1v3s_won',np.sum),
                        total_1v4s=('1v4s',np.sum),
                        total_1v4s_won=('1v4s_won',np.sum),
                        total_rnds=('rounds_played', np.sum))

    player_summary['fk%'] = player_summary['total_fk']/player_summary['total_rnds'] * 100
    player_summary['fd%'] = player_summary['total_fd']/player_summary['total_rnds'] * 100

    # Calculating clutche% in 1v2 + 1v3

    player_summary['clutch%'] = (player_summary['total_1v2s_won'] + player_summary['total_1v3s_won']) / \
                                (player_summary['total_1v2s'] + player_summary['total_1v3s']) * 100


    ## Last alive

    player_summary['last_alive'] = player_summary['total_1v2s'] + player_summary['total_1v3s'] + player_summary['total_1v4s']

    return player_summary.round(1).sort_values('acs',ascending=False)
    


In [13]:
eap_summary = player_summary(eap_df).drop(columns=['total_1v4s','total_1v4s_won','judge_kills'])

eap_summary['last_alive%'] = eap_summary['last_alive'] / eap_summary['total_rnds'] * 100

eap_summary[eap_summary['total_rnds'] >= 100].round(2).sort_values('last_alive%',ascending=False)

,hs,op_kills,acs,kast,total_fk,total_fd,total_1v2s,total_1v2s_won,total_1v3s,total_1v3s_won,total_rnds,fk%,fd%,clutch%,last_alive,last_alive%
name,,,,,,,,,,,,,,,,
f a b i o,23.9,0,284.3,77.2,43,22,6,2,15,1,267,16.1,8.2,14.3,29,10.86
PAYGE,20.0,7,219.3,76.8,29,33,16,3,15,1,381,7.6,8.7,12.9,40,10.50
W a f f l e,28.7,0,182.2,70.0,31,23,15,1,12,0,366,8.5,6.3,3.7,38,10.38
Kyrigan,23.8,4,215.0,75.5,38,35,16,5,8,1,345,11.0,10.1,25.0,27,7.83
StrawBerry,26.2,15,238.1,71.2,50,52,4,2,6,0,323,15.5,16.1,20.0,16,4.95
Opticss,27.4,0,135.3,71.2,11,39,7,1,4,0,387,2.8,10.1,9.1,17,4.39


In [14]:
tnl_df = performances_df[performances_df['team'] == 'Team Name Loading'].merge(maps_df)

## Replace 'tnl wzrd' with Neeks
tnl_df.replace('TNL wzrd', 'Neeks', inplace=True)

tnl_summary = player_summary(tnl_df).drop(columns=['total_1v4s','total_1v4s_won','judge_kills'])


tnl_summary['last_alive%'] = tnl_summary['last_alive'] / tnl_summary['total_rnds'] * 100
tnl_summary.drop(columns='last_alive', inplace=True)

tnl_summary[tnl_summary['total_rnds'] >= 100].round(2).sort_values('last_alive%',ascending=False)

,hs,op_kills,acs,kast,total_fk,total_fd,total_1v2s,total_1v2s_won,total_1v3s,total_1v3s_won,total_rnds,fk%,fd%,clutch%,last_alive%
name,,,,,,,,,,,,,,,
Allora,23.2,0,230.6,78.8,10,8,7,3,8,1,175,5.7,4.6,26.7,9.14
Neeks,30.0,7,289.4,85.0,35,16,11,3,5,2,245,14.3,6.5,31.2,8.57
TNL Bubbaaah,13.3,1,166.7,78.6,4,9,5,3,3,0,133,3.0,6.8,37.5,8.27
TNL Strober,24.7,0,223.3,76.4,25,26,3,0,5,1,175,14.3,14.9,12.5,6.86
TNL Divinity,12.7,0,256.5,79.6,24,29,3,1,3,1,233,10.3,12.4,33.3,6.01
gkjfldjgdfljlkgj,34.1,0,328.0,82.9,17,9,4,1,1,0,100,17.0,9.0,20.0,6.00
TNL Cano,13.6,0,134.7,75.0,4,5,1,0,2,0,144,2.8,3.5,0.0,5.56
TNL Devil,15.8,11,263.5,83.2,34,23,3,0,1,0,136,25.0,16.9,0.0,4.41
TNL Yoanks28,20.5,0,180.8,82.9,4,9,2,0,1,0,102,3.9,8.8,0.0,3.92
